In [1]:
import pandas as pd
import sqlite3

In [2]:
connect = sqlite3.connect('../data/checking-logs.sqlite')

In [3]:
schema_deadlines = pd.io.sql.read_sql('PRAGMA table_info(deadlines)', connect)
schema_deadlines

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [4]:
deadlines_head = pd.io.sql.read_sql('select * from deadlines limit 10', connect)
deadlines_head

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


## найти среди всех пользователей минимальное значение дельты между первым коммитом пользователя и крайним сроком соответствующей лаборатории

In [5]:
query = """
    SELECT t.uid,
        (unixepoch(t.first_commit_ts) - (d.deadlines)) / 3600 AS difference
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname != "project1"
    ORDER BY difference LIMIT 1;

"""

df_min = pd.io.sql.read_sql(query, connect)
df_min

,uid,difference
0,user_30,-202


In [6]:
query2 = """
    SELECT t.uid,
        (unixepoch(t.first_commit_ts) - (d.deadlines)) / 3600 AS difference
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname != "project1"
    ORDER BY difference DESC LIMIT 1;

"""

df_max = pd.io.sql.read_sql(query2, connect)
df_max

,uid,difference
0,user_25,-2


In [7]:
query3 = """
    SELECT 
        AVG((unixepoch(t.first_commit_ts) - (d.deadlines)) / 3600) AS 'AVG(diff)'
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname != "project1"

"""

df_avg = pd.io.sql.read_sql(query3, connect)
df_avg

,AVG(diff)
0,-89.125


In [12]:
query4 = """
    WITH diffs AS (
        SELECT
            t.uid,
            AVG((unixepoch(t.first_commit_ts) - (d.deadlines)) / 3600) AS avg_diff
        FROM 
            test t
        JOIN 
            deadlines d ON t.labname = d.labs
        WHERE 
            t.labname != 'project1'
        GROUP BY t.uid
    ),
    views AS (
        SELECT
            uid,
            COUNT(*) AS pageviews
            FROM pageviews
            GROUP BY uid
    )
    SELECT
        diffs.uid,
        diffs.avg_diff,
        views.pageviews
    FROM diffs
    JOIN views ON diffs.uid = views.uid
"""

views_diff = pd.io.sql.read_sql(query4, connect)
cor = views_diff[['avg_diff', 'pageviews']].corr()
cor

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [13]:
connect.close()